In [47]:
import pandas as pd
import numpy as np
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

In [48]:
train_input=  pd.read_csv("Credit_Risk_Train_data.csv")
test_input = pd.read_csv("Credit_Risk_Validate_data.csv")

In [49]:
print(train_input.columns)
print(test_input.columns)

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')
Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'outcome'],
      dtype='object')


In [50]:
# the last column has a different name in both;
#lets make the names same.and then merge them together
# so that we can fill the missing values simultenously
test_input.rename(columns={"outcome": "Loan_Status"},inplace=True)

In [51]:
data_all=pd.concat([train_input,test_input],axis=0)
data_all.shape

(981, 13)

In [52]:
data_all.reset_index(inplace=True,drop=True)
#reset index esle merging will have issues,
print(data_all.tail())

      Loan_ID Gender Married Dependents     Education Self_Employed  \
976  LP002971   Male     Yes         3+  Not Graduate           Yes   
977  LP002975   Male     Yes          0      Graduate            No   
978  LP002980   Male      No          0      Graduate            No   
979  LP002986   Male     Yes          0      Graduate            No   
980  LP002989   Male      No          0      Graduate           Yes   

     ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
976             4009             1777.0       113.0             360.0   
977             4158              709.0       115.0             360.0   
978             3250             1993.0       126.0             360.0   
979             5000             2393.0       158.0             360.0   
980             9200                0.0        98.0             180.0   

     Credit_History Property_Area Loan_Status  
976             1.0         Urban           Y  
977             1.0         Urban     

In [53]:
data_all.isnull().sum() #gives the missing values of all columns

Loan_ID               0
Gender               24
Married               3
Dependents           25
Education             0
Self_Employed        55
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [54]:
data_all.shape #read the description of each column from the word document

(981, 13)

In [16]:
#before proceeding to model building, lets fill the missing vlaues

In [55]:
Counter(data_all['Gender'])

Counter({'Male': 775, 'Female': 182, nan: 24})

In [56]:
#Lets fill them by Male
#these rows are null for gender
#Lets fill them with Model of Gender i.e Male
gender_null= data_all[data_all['Gender'].isnull()].index.tolist()
print(gender_null)

[23, 126, 171, 188, 314, 334, 460, 467, 477, 507, 576, 588, 592, 636, 665, 720, 752, 823, 845, 859, 893, 910, 917, 932]


In [ ]:
gender_null_M=gender_null[:12]
gender_null_F= gender_null[12:]
data_all['Gender'].iloc[gender_null_M]="Male"
data_all['Gender'].iloc[gender_null_F]="Female"

In [57]:
data_all['Gender'].iloc[gender_null]="Male"

In [58]:
#check if filed
print(sum(data_all['Gender'].isnull()))  #oky done
Counter(data_all['Gender'])

0


Counter({'Male': 799, 'Female': 182})

In [25]:
#Lets fill Married now
print(Counter(data_all['Married'])) #most are married

Counter({'Yes': 631, 'No': 347, nan: 3})


In [59]:
married_null=data_all[data_all['Married'].isnull()].index.tolist()
married_null

[104, 228, 435]

In [60]:
data_all['Married'].iloc[married_null]="Yes"

In [61]:
data_all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents           25
Education             0
Self_Employed        55
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [29]:
Counter(data_all['Dependents'])

Counter({'0': 545, '1': 160, '2': 160, '3+': 91, nan: 25})

In [62]:
#lets fill Dependents wrt marriage
pd.crosstab(data_all['Married'],data_all['Dependents'].isnull())

Dependents,False,True
Married,,
No,338,9
Yes,618,16


In [63]:
pd.crosstab(data_all['Dependents'],data_all['Married'])

Married,No,Yes
Dependents,,
0,276,269
1,36,124
2,14,146
3+,12,79


In [33]:
#for the bachelors,lets fill the missing dependents as 0
#lets find the index of all rows with Dependents missing and married No
bachelor_nulldependent=data_all[(data_all['Married']=="No")&(data_all['Dependents'].isnull())].index.tolist()
print(bachelor_nulldependent)

[293, 332, 355, 597, 684, 752, 879, 916, 926]


In [64]:
data_all['Dependents'].iloc[bachelor_nulldependent]='0'

In [65]:
Counter(data_all['Dependents'])

Counter({'0': 554, '1': 160, '2': 160, '3+': 91, nan: 16})

In [66]:
#for the remaining 16 missing dependents,
#let see how many dependents Male & Female have
pd.crosstab(data_all['Gender'],data_all['Dependents'])

Dependents,0,1,2,3+
Gender,,,,
Female,127,32,13,9
Male,427,128,147,82


In [ ]:
pd.crosstab(data_all['Gender'],data_all['Dependents'].isnull())

In [67]:
#so female have less dependents
#lets see the gender of the 16 missing dependents
data_all['Gender'].iloc[data_all[data_all['Dependents'].isnull()].index.tolist()]

102      Male
104      Male
120      Male
226      Male
228      Male
301      Male
335      Male
346      Male
435    Female
517      Male
571      Male
660      Male
725      Male
816      Male
861      Male
865      Male
Name: Gender, dtype: object

In [68]:
pd.crosstab((data_all['Gender']=='Male')&(data_all['Married']=='Yes'),data_all['Dependents'])

Dependents,0,1,2,3+
row_0,,,,
False,318,48,23,15
True,236,112,137,76


In [69]:
#lets fill the #dependent with1

data_all['Dependents'].iloc[data_all[data_all['Dependents'].isnull()].index.tolist()]="1"


In [70]:
data_all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed        55
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [71]:
Counter(data_all['Self_Employed'])

Counter({'No': 807, 'Yes': 119, nan: 55})

In [72]:
self_emp_null=data_all[data_all['Self_Employed'].isnull()].index.tolist()

In [73]:
#fill missing selfemployed with NO
data_all['Self_Employed'].iloc[self_emp_null]="No"

In [74]:
data_all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [75]:
#ro check if any row with both LoanAMount and loan_amount_term as NAN
pd.crosstab(data_all['LoanAmount'].isnull(),
           data_all['Loan_Amount_Term'].isnull())

Loan_Amount_Term,False,True
LoanAmount,,
False,934,20
True,27,0


In [76]:
pd.crosstab(data_all['LoanAmount'].isnull(),data_all['Loan_Amount_Term'])

Loan_Amount_Term,6.0,12.0,36.0,60.0,84.0,120.0,180.0,240.0,300.0,350.0,360.0,480.0
LoanAmount,,,,,,,,,,,,
False,1,2,3,3,7,4,64,7,20,1,800,22
True,0,0,0,0,0,0,2,1,0,0,23,1


In [77]:
data_all.groupby(data_all['Loan_Amount_Term'])['LoanAmount'].mean()

Loan_Amount_Term
6.0       95.000000
12.0     185.500000
36.0     117.666667
60.0     139.666667
84.0     121.142857
120.0     36.750000
180.0    131.125000
240.0    128.857143
300.0    166.250000
350.0    133.000000
360.0    144.420000
480.0    137.181818
Name: LoanAmount, dtype: float64

In [79]:
#Lets fill the missing values in LoanAmount
#with the mean of the respective Loan_Term
#we see that 180&240 has the almost same Loan amount 128-131
# & 360 has high i.e 144
# so lets fill only 360 by 144
# and all remaining by 130

data_all['LoanAmount'][(data_all['LoanAmount'].isnull())
                      &(data_all['Loan_Amount_Term']==360)]=144
data_all['LoanAmount'][(data_all['LoanAmount'].isnull())
                      &(data_all['Loan_Amount_Term']==480)]=137


In [80]:
data_all['LoanAmount'][(data_all['LoanAmount'].isnull())]=130

In [81]:
#lets fill LoanAmount Term
(data_all['Loan_Amount_Term']).value_counts()

360.0    823
180.0     66
480.0     23
300.0     20
240.0      8
84.0       7
120.0      4
36.0       3
60.0       3
12.0       2
350.0      1
6.0        1
Name: Loan_Amount_Term, dtype: int64

In [82]:
#lets fill the Loan Tenure by the mode i,e 360
data_all['Loan_Amount_Term'][data_all['Loan_Amount_Term'].isnull()]=360

In [83]:
data_all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [85]:
data_all['Credit_History'].value_counts()

1.0    754
0.0    148
Name: Credit_History, dtype: int64

In [87]:
pd.crosstab(data_all['Gender'],data_all['Credit_History'])
#Gender makes no difference

Credit_History,0.0,1.0
Gender,,
Female,30,135
Male,118,619


In [88]:
pd.crosstab(data_all['Self_Employed'],data_all['Credit_History']) #self_employed makes no diff

Credit_History,0.0,1.0
Self_Employed,,
No,134,658
Yes,14,96


In [90]:
pd.crosstab(data_all['Education'],data_all['Credit_History'])

Credit_History,0.0,1.0
Education,,
Graduate,106,596
Not Graduate,42,158


In [92]:
pd.crosstab(data_all['Married'],data_all['Credit_History']) #no diff

Credit_History,0.0,1.0
Married,,
No,56,263
Yes,92,491


In [95]:
data_all['Credit_History'][data_all['Credit_History'].isnull()]=1

In [96]:
data_all.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [97]:
data_all.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,144.0,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [98]:
data_all_new = pd.get_dummies(data_all.drop(['Loan_ID'],axis=1),drop_first=True)

In [99]:
data_all_new.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_Y
0,5849,0.0,144.0,360.0,1.0,1,0,0,0,0,0,0,0,1,1
1,4583,1508.0,128.0,360.0,1.0,1,1,1,0,0,0,0,0,0,0
2,3000,0.0,66.0,360.0,1.0,1,1,0,0,0,0,1,0,1,1
3,2583,2358.0,120.0,360.0,1.0,1,1,0,0,0,1,0,0,1,1
4,6000,0.0,141.0,360.0,1.0,1,0,0,0,0,0,0,0,1,1


In [113]:
X = data_all_new.drop(['Loan_Status_Y'],axis=1)
y = data_all_new['Loan_Status_Y']

In [114]:
X.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban
0,5849,0.0,144.0,360.0,1.0,1,0,0,0,0,0,0,0,1
1,4583,1508.0,128.0,360.0,1.0,1,1,1,0,0,0,0,0,0
2,3000,0.0,66.0,360.0,1.0,1,1,0,0,0,0,1,0,1
3,2583,2358.0,120.0,360.0,1.0,1,1,0,0,0,1,0,0,1
4,6000,0.0,141.0,360.0,1.0,1,0,0,0,0,0,0,0,1


In [102]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: Loan_Status_Y, dtype: uint8

### Train Test Split

In [104]:
from sklearn.model_selection import train_test_split

In [105]:
X_train,X_test,y_train,y_test = train_test_split(x,y)

In [106]:
X_train.shape

(735, 14)

In [107]:
X_test.shape

(246, 14)

### DataProcessing

In [109]:
from sklearn.preprocessing import StandardScaler

In [110]:
scaler = StandardScaler()

In [115]:
#fit only to the training data
scaler.fit(X)

StandardScaler()

In [116]:
# now apply the transformations to the data:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [117]:
X_train[:5]

array([[ 0.4076112 , -0.5895062 , -0.88455397,  0.2705276 ,  0.42151046,
         0.47726799,  0.73980985,  2.13865931, -0.44145701, -0.31976115,
        -0.53452248,  2.6914118 , -0.74311183,  1.36690199],
       [ 6.1170019 , -0.5895062 ,  5.99505053, -2.52237248, -2.37242036,
         0.47726799,  0.73980985, -0.46758266, -0.44145701,  3.12733429,
        -0.53452248, -0.37155221,  1.34569248, -0.73158135],
       [-0.47886487,  0.26903924,  0.50447094,  0.2705276 , -2.37242036,
         0.47726799,  0.73980985, -0.46758266, -0.44145701, -0.31976115,
         1.87082869, -0.37155221, -0.74311183,  1.36690199],
       [-0.07550596,  0.72904002,  0.01962262,  0.2705276 ,  0.42151046,
         0.47726799,  0.73980985, -0.46758266, -0.44145701, -0.31976115,
         1.87082869, -0.37155221,  1.34569248, -0.73158135],
       [-0.04037017, -0.5895062 , -0.22935354,  0.2705276 ,  0.42151046,
         0.47726799, -1.35169869, -0.46758266, -0.44145701, -0.31976115,
        -0.53452248, -0.37

### Training the model using SVM

In [118]:
from sklearn import svm

In [119]:
clf = svm.SVC(kernel='linear',C=1.0)
clf.fit(X_train,y_train)

SVC(kernel='linear')

In [120]:
predictions = clf.predict(X_test)

In [121]:
from sklearn.metrics import classification_report,confusion_matrix

In [122]:
print(confusion_matrix(y_test,predictions))

[[ 35  32]
 [  3 176]]


In [123]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.92      0.52      0.67        67
           1       0.85      0.98      0.91       179

    accuracy                           0.86       246
   macro avg       0.88      0.75      0.79       246
weighted avg       0.87      0.86      0.84       246

